Agent version example

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

# Set environment
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
model = ChatOpenAI(model="gpt-3.5-turbo")
embedding = OpenAIEmbeddings()

In [4]:
from langchain_community.vectorstores import Chroma
import document_handler

# https://python.langchain.com/docs/integrations/vectorstores/chroma

chroma_collection_name = "LangChainCollection"

import chromadb
new_client = chromadb.EphemeralClient()

vectorstore_initialize = Chroma.from_documents(
    document_handler.processed_texts,
    embedding=embedding,
    collection_name=chroma_collection_name,
    client=new_client,
)

vectorstore = Chroma(
    client=new_client,
    collection_name=chroma_collection_name,
    embedding_function=embedding,
)
retriever = vectorstore.as_retriever()

In [5]:
docs = vectorstore_initialize.similarity_search("What is Chocolate?")
print(docs)
docs = vectorstore.similarity_search("What is Chocolate?")
print(docs)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(page_content='Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolates are often used in confectionery and desserts, and they can be found in a wide range of products, including bars, truffles, candies, and baked goods. Chocolate is enjoyed worldwide and is often associated with indulgence and celebration. Additionally, cocoa has been linked to various potential health benefits, such as antioxidants and mood enhancement, when consumed in moderation.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white 

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


[Document(page_content='Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolates are often used in confectionery and desserts, and they can be found in a wide range of products, including bars, truffles, candies, and baked goods. Chocolate is enjoyed worldwide and is often associated with indulgence and celebration. Additionally, cocoa has been linked to various potential health benefits, such as antioxidants and mood enhancement, when consumed in moderation.', metadata={'source': 'test_data\\Chocolate.txt'}), Document(page_content='Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white 

In [6]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder
)
from langchain_core.output_parsers import StrOutputParser
from langchain.agents import AgentExecutor, tool
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [48]:
from langchain.memory import ConversationSummaryBufferMemory
conversational_memory = ConversationSummaryBufferMemory(
    k=4,
    max_token_limit=64,
    llm=model,
    memory_key = "chat_history",
    return_messages=True
)

In [49]:
# Prompt

system_message_template = (
    "You are a helpful assistant who helps answer questions. Answer only the facts based on the context. "
    "Your goal is to provide accurate and relevant answers based on the facts in the provided context. "
    "Make sure to reference the above source documents appropriately and avoid making assumptions or adding personal opinions. "
    "Emphasize the use of facts from the provided source documents. "
    "Instruct the model to use source name for each fact used in the response. "
    "Avoid generating speculative or generalized information. "
    "Use square brackets to reference the source, e.g. [info1.txt]. "
    "Do not combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\n"
    "Here is how you should answer every question:\n"
        "-Look for relevant information in the above source documents to answer the question.\n"
        "-If the source document does not include the exact answer, please respond with relevant information from the data in the response along with citation. You must include a citation to each document referenced.\n"
        "-If you cannot find answer in below sources, respond with I am not sure. Do not provide personal opinions or assumptions and do not include citations.\n"
        "-If you use any information in the context, include the index(starts at 1) of the statement as citation in your answer\n"
    "At the end of your response:\n" 
    "1. Add key words from the paragraphs. \n"
    "2. Suggest a further question that can be answered by the paragraphs provided. \n"
    "3. Create a source list of source name, author name, and a link for each document you cited.\n"
    "{context}"

)

MEMORY_KEY = "chat_history"

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message_template),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("human", "{question}"),
        # MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


In [50]:
from langchain.tools.retriever import create_retriever_tool
# tools
retriever_tool = create_retriever_tool(
    retriever,
    "search_docs",
    "Searches and returns documents for information to answer the question",
)

#Below for debugging
@tool
def retrieve_documents_tool(query: str) -> str:
    """Searches for relevant documents that may contain the answer to the query."""
    docs = retriever.get_relevant_documents(query)[:3]
    # return docs
    texts = [doc.page_content for doc in docs]
    texts_merged = '\n\n'.join(texts)
    print("Retrieving documents")
    print(texts_merged)
    return texts_merged


tools = [retriever_tool]

In [51]:
# Set llm chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# agent = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | final_prompt
#     | model
#     | StrOutputParser()
# )
from langchain.agents.conversational_chat.base import ConversationalChatAgent

agent = ConversationalChatAgent.from_llm_and_tools(llm=model, tools=tools, prompt=final_prompt)

In [52]:
chat_history = []

In [53]:
agent_executor = AgentExecutor(tools=tools, agent=agent, verbose=True, memory = conversational_memory)

In [54]:
query = "What is chocolate?"
print(agent_executor.invoke({"input":query}))



> Entering new AgentExecutor chain...
{
  "action": "search_docs",
  "action_input": "What is chocolate?"
}

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3



Observation: Chocolate is a sweet, usually brown, food product made from cocoa beans, which are the seeds of the cacao tree. The process of making chocolate involves harvesting and fermenting cacao beans, drying them, and then roasting and grinding them to produce cocoa mass. This cocoa mass is then further processed to extract cocoa solids and cocoa butter.

Chocolates are often used in confectionery and desserts, and they can be found in a wide range of products, including bars, truffles, candies, and baked goods. Chocolate is enjoyed worldwide and is often associated with indulgence and celebration. Additionally, cocoa has been linked to various potential health benefits, such as antioxidants and mood enhancement, when consumed in moderation.

Chocolate comes in various forms, such as dark chocolate, milk chocolate, and white chocolate. Dark chocolate contains a higher percentage of cocoa solids and less sugar, giving it a more intense and bitter flavor. Milk chocolate, on the othe